**We will collect text data using Twitter API.**

In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

## Initializing Twitter API

In [2]:
consumer_key = "DreBJqr1OxQZGAuMJIS3kaeTL"
consumer_secret = "5dMdE5ptm468W5o5vQf48AMShNnYGQDvtRkuhcIyycz4wBGKIj"
access_token = "1322035865615556609-z6ZRy7ZJFgNNNleEE4u6zRZ7pNTvTm"
access_token_secret = "Ty0OUxG0ng1YvqihRiDFmhAvOH7j2QhNBbUzWbBi07CH9"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "Narendra Modi Prime Minister"

In [5]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

# for tweet in Tweets:
#     print(tweet,"\n")
#     print(".....................................\n")

## Creating Dataframe by fetching data using tweepy

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=[query] , file_name='my_tweets')

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @stpiindia: The grandest tech event Bengalu...,Ajay Shrivastava,6427,222,Mohali Punjab,False,0,220,2020-11-19 06:07:33
1,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",Ajay Shrivastava,6427,222,Mohali Punjab,False,0,213,2020-11-19 06:06:41
2,RT @stpiindia: The grandest tech event Bengalu...,STPI Lucknow,13454,1196,"Gomti Nagar, Lucknow",False,0,220,2020-11-19 06:06:37
3,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",STPI Lucknow,13454,1196,"Gomti Nagar, Lucknow",False,0,213,2020-11-19 06:06:30
4,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",STPI Pune,6384,2239,"Plot No. P1, MIDC, Rajiv Gandhi Infotech Park,...",False,0,213,2020-11-19 06:06:24


## Cleaning Tweets and analyzing using TextBlob

In [10]:
from textblob import TextBlob

In [11]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [12]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [13]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [14]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,RT @stpiindia: The grandest tech event Bengalu...,Ajay Shrivastava,6427,222,Mohali Punjab,False,0,220,2020-11-19 06:07:33,RT The grandest tech event Bengaluru Tech Summ...
1,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",Ajay Shrivastava,6427,222,Mohali Punjab,False,0,213,2020-11-19 06:06:41,RT Shri Narendra Modi Hon ble Prime Minister o...
2,RT @stpiindia: The grandest tech event Bengalu...,STPI Lucknow,13454,1196,"Gomti Nagar, Lucknow",False,0,220,2020-11-19 06:06:37,RT The grandest tech event Bengaluru Tech Summ...
3,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",STPI Lucknow,13454,1196,"Gomti Nagar, Lucknow",False,0,213,2020-11-19 06:06:30,RT Shri Narendra Modi Hon ble Prime Minister o...
4,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",STPI Pune,6384,2239,"Plot No. P1, MIDC, Rajiv Gandhi Infotech Park,...",False,0,213,2020-11-19 06:06:24,RT Shri Narendra Modi Hon ble Prime Minister o...


In [15]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @stpiindia: The grandest tech event Bengalu...,Ajay Shrivastava,6427,222,Mohali Punjab,False,0,220,2020-11-19 06:07:33,RT The grandest tech event Bengaluru Tech Summ...,Neutral
1,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",Ajay Shrivastava,6427,222,Mohali Punjab,False,0,213,2020-11-19 06:06:41,RT Shri Narendra Modi Hon ble Prime Minister o...,Neutral
2,RT @stpiindia: The grandest tech event Bengalu...,STPI Lucknow,13454,1196,"Gomti Nagar, Lucknow",False,0,220,2020-11-19 06:06:37,RT The grandest tech event Bengaluru Tech Summ...,Neutral
3,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",STPI Lucknow,13454,1196,"Gomti Nagar, Lucknow",False,0,213,2020-11-19 06:06:30,RT Shri Narendra Modi Hon ble Prime Minister o...,Neutral
4,"RT @stpiindia: Shri Narendra Modi, Hon’ble Pri...",STPI Pune,6384,2239,"Plot No. P1, MIDC, Rajiv Gandhi Infotech Park,...",False,0,213,2020-11-19 06:06:24,RT Shri Narendra Modi Hon ble Prime Minister o...,Neutral


In [16]:
df[df['User_verified'] == True].count()

Tweets                 37
User                   37
User_statuses_count    37
user_followers         37
User_location          37
User_verified          37
fav_count              37
rt_count               37
tweet_date             37
clean_tweet            37
Sentiment              37
dtype: int64

In [17]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @stpiindia: Shri Narendra Modi, Hon’ble Prime Minister of India will inaugurate the largest tech event Bengaluru Tech Summit 2020 virtua…

Clean tweet:
 RT Shri Narendra Modi Hon ble Prime Minister of India will inaugurate the largest tech event Bengaluru Tech Summit 2020 virtua

Sentiment of the tweet:
 Neutral


In [18]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Through technology, we have enhanced human dignity. Millions of farmers received monetary support in one click. At… https://t.co/nYxaaetZuc

Clean tweet:
 Through technology we have enhanced human dignity Millions of farmers received monetary support in one click At

Sentiment of the tweet:
 Neutral


In [19]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 #PMModiAtBTS2020
Dr Narayan expressed happiness over the gesture shown by Prime Minister Narendra Modi to inaugurat… https://t.co/OYATjIaFpP

Clean tweet:
 PMModiAtBTS2020 Dr Narayan expressed happiness over the gesture shown by Prime Minister Narendra Modi to inaugurat

Sentiment of the tweet:
 Positive


In [20]:
df[df.Sentiment == 'Positive'].shape[0]

302

In [21]:
df[df.Sentiment == 'Neutral'].shape[0]

675

In [22]:
df[df.Sentiment == 'Negative'].shape[0]

23